# Automatyczne Dekodowanie i Przenoszenie Części Wideo

Ten notatnik automatyzuje proces:
1. Dekodowania VVC dla każdej części wideo (part_1 do part_5)
2. **Przenoszenia** (wycinania) zdekodowanych danych do `/mnt/d/data_mgr/decoded_new`

Każda część będzie przetwarzana sekwencyjnie - dekodowanie, potem przeniesienie.
**Uwaga:** Pliki będą przenoszone (nie kopiowane), więc folder źródłowy zostanie usunięty po przeniesieniu.

## 1. Import Bibliotek

In [6]:
import subprocess
import shutil
from pathlib import Path
import time
from datetime import datetime

## 2. Konfiguracja

In [7]:
# Ścieżki i parametry
WORKSPACE_DIR = Path("/home/karol/mgr/new_PC")
DESTINATION_DIR = Path("/mnt/d/data_mgr/decoded_new")
CPU_COUNT = 12
NUM_PARTS = 10

# Sprawdzenie czy folder docelowy istnieje
DESTINATION_DIR.mkdir(parents=True, exist_ok=True)
print(f"Folder docelowy: {DESTINATION_DIR}")
print(f"Liczba części do przetworzenia: {NUM_PARTS}")
print(f"Liczba rdzeni CPU: {CPU_COUNT}")

Folder docelowy: /mnt/d/data_mgr/decoded_new
Liczba części do przetworzenia: 10
Liczba rdzeni CPU: 12


## 3. Funkcje Pomocnicze

In [8]:
def run_decode_task(part_num):
    """Uruchamia dekodowanie dla danej części."""
    task_file = f"TASKS_DEC{part_num}"
    done_file = f"done_dec{part_num}"
    undone_file = f"undone_dec{part_num}"
    
    print(f"\n{'='*60}")
    print(f"CZĘŚĆ {part_num}: Rozpoczęcie dekodowania")
    print(f"{'='*60}")
    print(f"Plik zadań: {task_file}")
    print(f"Czas rozpoczęcia: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    start_time = time.time()
    
    # Uruchom pool_work.py
    cmd = [
        "python3", "bin/pool_work.py",
        task_file, done_file, undone_file,
        "--cpu-count", str(CPU_COUNT)
    ]
    
    try:
        # Uruchom bez capture_output - zobaczysz wszystkie printy na żywo
        result = subprocess.run(
            cmd,
            cwd=WORKSPACE_DIR,
            check=True
        )
        
        elapsed_time = time.time() - start_time
        print(f"\n{'='*60}")
        print(f"✓ Dekodowanie zakończone pomyślnie")
        print(f"Czas trwania: {elapsed_time/60:.1f} minut")
        print(f"{'='*60}")
        return True
        
    except subprocess.CalledProcessError as e:
        print(f"✗ Błąd podczas dekodowania:")
        print(f"  Kod wyjścia: {e.returncode}")
        return False


def move_decoded_files(part_num):
    """Przenosi (wycina) zdekodowane pliki do folderu docelowego."""
    source_dir = WORKSPACE_DIR / "videos" / "decoded" / f"part_{part_num}"
    
    print(f"\n{'='*60}")
    print(f"CZĘŚĆ {part_num}: Przenoszenie plików")
    print(f"{'='*60}")
    print(f"Źródło: {source_dir}")
    print(f"Cel: {DESTINATION_DIR}")
    
    if not source_dir.exists():
        print(f"✗ Folder źródłowy nie istnieje: {source_dir}")
        return False
    
    start_time = time.time()
    
    try:
        # Policz pliki przed przeniesieniem
        num_files = sum(1 for _ in source_dir.rglob("*") if _.is_file())
        
        # Przenieś wszystkie pliki i podfoldery
        for item in source_dir.rglob("*"):
            if item.is_file():
                # Zachowaj strukturę folderów
                relative_path = item.relative_to(source_dir)
                dest_path = DESTINATION_DIR / relative_path
                
                # Utwórz folder docelowy jeśli nie istnieje
                dest_path.parent.mkdir(parents=True, exist_ok=True)
                
                # Przenieś plik (zamiast kopiować)
                shutil.move(str(item), str(dest_path))
        
        # Usuń pusty folder źródłowy
        shutil.rmtree(source_dir)
        
        elapsed_time = time.time() - start_time
        
        print(f"✓ Przenoszenie zakończone pomyślnie")
        print(f"Przeniesiono plików: {num_files}")
        print(f"✓ Usunięto folder źródłowy: {source_dir}")
        print(f"Czas trwania: {elapsed_time:.1f} sekund")
        print(f"{'='*60}")
        return True
        
    except Exception as e:
        print(f"✗ Błąd podczas przenoszenia:")
        print(f"  {str(e)}")
        return False

## 4. Główny Proces - Dekodowanie i Przenoszenie Wszystkich Części

In [9]:
# Statystyki
results = []
total_start_time = time.time()

print(f"\n{'#'*60}")
print(f"# ROZPOCZĘCIE PRZETWARZANIA WSZYSTKICH CZĘŚCI")
print(f"# Czas rozpoczęcia: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"{'#'*60}\n")

for part_num in range(1, NUM_PARTS + 1):
    part_start = time.time()
    
    # 1. Dekodowanie
    decode_success = run_decode_task(part_num)
    
    # 2. Przenoszenie (tylko jeśli dekodowanie się powiodło)
    move_success = False
    if decode_success:
        move_success = move_decoded_files(part_num)
    
    # Zapisz wyniki
    part_time = time.time() - part_start
    results.append({
        'part': part_num,
        'decode_success': decode_success,
        'move_success': move_success,
        'time': part_time
    })
    
    print(f"\nCzęść {part_num} - Czas całkowity: {part_time/60:.1f} minut")
    print(f"Status: {'✓ SUKCES' if (decode_success and move_success) else '✗ BŁĄD'}")

# Podsumowanie
total_time = time.time() - total_start_time

print(f"\n{'#'*60}")
print(f"# PODSUMOWANIE")
print(f"{'#'*60}")
print(f"Całkowity czas: {total_time/60:.1f} minut ({total_time/3600:.2f} godzin)")
print(f"\nWyniki dla poszczególnych części:")

for r in results:
    status = "✓" if (r['decode_success'] and r['move_success']) else "✗"
    print(f"  Część {r['part']}: {status} "
          f"(dekodowanie: {'✓' if r['decode_success'] else '✗'}, "
          f"przenoszenie: {'✓' if r['move_success'] else '✗'}) "
          f"- {r['time']/60:.1f} min")

successful = sum(1 for r in results if r['decode_success'] and r['move_success'])
print(f"\nPomyślnie przetworzono: {successful}/{NUM_PARTS} części")
print(f"Zakończono: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


############################################################
# ROZPOCZĘCIE PRZETWARZANIA WSZYSTKICH CZĘŚCI
# Czas rozpoczęcia: 2025-11-02 03:06:06
############################################################


CZĘŚĆ 1: Rozpoczęcie dekodowania
Plik zadań: TASKS_DEC1
Czas rozpoczęcia: 2025-11-02 03:06:06
Spawning processes...
Spawning bin/decode_data.sh videos/encoded/part_1/FourPeople_1280x720_60_AI_QP28_ALF0_DB0_SAO0.vvc videos/decoded/part_1...
Spawning bin/decode_data.sh videos/encoded/part_1/FourPeople_1280x720_60_AI_QP28_ALF0_DB0_SAO1.vvc videos/decoded/part_1...
Spawning bin/decode_data.sh videos/encoded/part_1/FourPeople_1280x720_60_AI_QP28_ALF0_DB1_SAO0.vvc videos/decoded/part_1...
Spawning bin/decode_data.sh videos/encoded/part_1/FourPeople_1280x720_60_AI_QP28_ALF0_DB1_SAO1.vvc videos/decoded/part_1...
Spawning bin/decode_data.sh videos/encoded/part_1/FourPeople_1280x720_60_AI_QP28_ALF1_DB0_SAO0.vvc videos/decoded/part_1...
Spawning bin/decode_data.sh videos/encoded/part_1/Fou

KeyboardInterrupt: 

## 5. Opcjonalnie - Sprawdzenie Zawartości Folderu Docelowego

In [ ]:
# Sprawdzenie ile plików zostało skopiowanych
if DESTINATION_DIR.exists():
    total_files = sum(1 for _ in DESTINATION_DIR.rglob("*") if _.is_file())
    
    # Policz pliki według rozszerzeń
    extensions = {}
    for file in DESTINATION_DIR.rglob("*"):
        if file.is_file():
            ext = file.suffix or 'no_extension'
            extensions[ext] = extensions.get(ext, 0) + 1
    
    print(f"Folder docelowy: {DESTINATION_DIR}")
    print(f"Całkowita liczba plików: {total_files}")
    print(f"\nPliki według rozszerzeń:")
    for ext, count in sorted(extensions.items(), key=lambda x: x[1], reverse=True):
        print(f"  {ext}: {count}")
    
    # Rozmiar folderu
    total_size = sum(f.stat().st_size for f in DESTINATION_DIR.rglob("*") if f.is_file())
    print(f"\nCałkowity rozmiar: {total_size / (1024**3):.2f} GB")
else:
    print(f"Folder {DESTINATION_DIR} nie istnieje!")

Folder docelowy: /mnt/d/data_mgr/decoded_new
Całkowita liczba plików: 0

Pliki według rozszerzeń:

Całkowity rozmiar: 0.00 GB
